In [104]:
import pandas as pd

In [105]:
df = pd.read_csv('/home/jensk/code/JensKlug/zillows_real_estate/raw_data/HouseTS.csv')

In [106]:
# Make sure it's sorted by city, zipcode, and date
df = df.sort_values(['city', 'zipcode', 'date']).reset_index(drop=True)

# Convert date to datetime if not already
df['date'] = pd.to_datetime(df['date'])


In [107]:
train = df[df.date.dt.year<2023]
train.shape

(809380, 39)

In [108]:
test = df[df.date.dt.year>2022]
test.shape

(74712, 39)

In [109]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 809380 entries, 0 to 884079
Data columns (total 39 columns):
 #   Column                        Non-Null Count   Dtype         
---  ------                        --------------   -----         
 0   date                          809380 non-null  datetime64[ns]
 1   median_sale_price             809380 non-null  float64       
 2   median_list_price             809380 non-null  float64       
 3   median_ppsf                   809380 non-null  float64       
 4   median_list_ppsf              809380 non-null  float64       
 5   homes_sold                    809380 non-null  float64       
 6   pending_sales                 809380 non-null  float64       
 7   new_listings                  809380 non-null  float64       
 8   inventory                     809380 non-null  float64       
 9   median_dom                    809380 non-null  float64       
 10  avg_sale_to_list              809380 non-null  float64       
 11  sold_above_list   

In [110]:
train.columns

Index(['date', 'median_sale_price', 'median_list_price', 'median_ppsf',
       'median_list_ppsf', 'homes_sold', 'pending_sales', 'new_listings',
       'inventory', 'median_dom', 'avg_sale_to_list', 'sold_above_list',
       'off_market_in_two_weeks', 'city', 'zipcode', 'year', 'bank', 'bus',
       'hospital', 'mall', 'park', 'restaurant', 'school', 'station',
       'supermarket', 'Total Population', 'Median Age', 'Per Capita Income',
       'Total Families Below Poverty', 'Total Housing Units', 'Median Rent',
       'Median Home Value', 'Total Labor Force', 'Unemployed Population',
       'Total School Age Population', 'Total School Enrollment',
       'Median Commute Time', 'price', 'city_full'],
      dtype='object')

In [111]:
# Group by city and zipcode for proper time series handling
grouped_train = train.groupby(['city', 'zipcode'])

In [112]:
grouped_test = test.groupby(['city', 'zipcode'])

In [113]:
type(grouped_train), type(train)

(pandas.core.groupby.generic.DataFrameGroupBy, pandas.core.frame.DataFrame)

In [114]:
#grouped_train_train.head(1) #With this command I get 6226 lines back which are the unique combinations of cities to zipcodes.
#train.head(1) #Here in contrast I just get one line back since it's a normal dataframe and not a groupby pandas object.

In [115]:
#Create a time series dataframe which will serve as training data for the model.
ts_train = pd.DataFrame()

In [116]:
ts_test = pd.DataFrame()

In [117]:
grouped_train['price']

In [118]:
# Create 3-month profitability target
ts_train['target_profitable_3m'] = (grouped_train['price'].shift(-3) > 1.02 * train['price']).astype(int)

numeric_column_names = train.select_dtypes(include='number').columns

for feature in numeric_column_names:
    for lag in range(1, 4):  # 1 to 6 months
        ts_train[f'{feature}_{lag}m'] = grouped_train[f'{feature}'].shift(lag)


/tmp/ipykernel_54040/36949176.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  ts_train[f'{feature}_{lag}m'] = grouped_train[f'{feature}'].shift(lag)
/tmp/ipykernel_54040/36949176.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  ts_train[f'{feature}_{lag}m'] = grouped_train[f'{feature}'].shift(lag)
/tmp/ipykernel_54040/36949176.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd

In [120]:
# Create 3-month profitability target
ts_test['target_profitable_3m'] = (grouped_test['price'].shift(-3) > 1.02 * test['price']).astype(int)

numeric_column_names = test.select_dtypes(include='number').columns

for feature in numeric_column_names:
    for lag in range(1, 4):  # 1 to 6 months
        ts_test[f'{feature}_{lag}m'] = grouped_test[f'{feature}'].shift(lag)

ts_test = ts_test.dropna().reset_index(drop=True)

/tmp/ipykernel_54040/2638047860.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  ts_test[f'{feature}_{lag}m'] = grouped_test[f'{feature}'].shift(lag)
/tmp/ipykernel_54040/2638047860.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  ts_test[f'{feature}_{lag}m'] = grouped_test[f'{feature}'].shift(lag)
/tmp/ipykernel_54040/2638047860.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using 

In [121]:
# Remove rows where target cannot be calculated (last 3 months for each location)
ts_train = ts_train.dropna().reset_index(drop=True)

In [122]:
print(f"Created features: {ts_train.columns.shape} - 1 target.")
print("Tagert column: target_profitable_3m")
print(f"Final dataset shape: {ts_train.shape}")

Created features: (109,) - 1 target.
Tagert column: target_profitable_3m
Final dataset shape: (790702, 109)


In [123]:
ts_train['target_profitable_3m'].value_counts()

target_profitable_3m
0    470137
1    320565
Name: count, dtype: int64

In [124]:
print(f"Created features: {ts_test.columns.shape} - 1 target.")
print("Tagert column: target_profitable_3m")
print(f"Final dataset shape: {ts_test.shape}")

Created features: (109,) - 1 target.
Tagert column: target_profitable_3m
Final dataset shape: (56034, 109)


In [125]:
#ts_train.columns.values

In [126]:
ts_train[ts_train.zipcode_1m==30002]

,target_profitable_3m,median_sale_price_1m,median_sale_price_2m,median_sale_price_3m,median_list_price_1m,median_list_price_2m,median_list_price_3m,median_ppsf_1m,median_ppsf_2m,median_ppsf_3m,...,Total School Age Population_3m,Total School Enrollment_1m,Total School Enrollment_2m,Total School Enrollment_3m,Median Commute Time_1m,Median Commute Time_2m,Median Commute Time_3m,price_1m,price_2m,price_3m
0,0,125500.0,61870.0,46550.0,217450.0,245000.0,217450.0,63.913043,40.723982,31.813674,...,5408.0,5408.0,5408.0,5408.0,2492.0,2492.0,2492.0,202681.309539,202421.064584,200773.999557
1,0,153000.0,125500.0,61870.0,189900.0,217450.0,245000.0,81.598080,63.913043,40.723982,...,5408.0,5408.0,5408.0,5408.0,2492.0,2492.0,2492.0,202998.603897,202681.309539,202421.064584
2,0,165500.0,153000.0,125500.0,154000.0,189900.0,217450.0,81.598080,81.598080,63.913043,...,5408.0,5408.0,5408.0,5408.0,2492.0,2492.0,2492.0,203781.903446,202998.603897,202681.309539
3,0,140250.0,165500.0,153000.0,145000.0,154000.0,189900.0,78.918228,81.598080,81.598080,...,5408.0,5408.0,5408.0,5408.0,2492.0,2492.0,2492.0,205012.252913,203781.903446,202998.603897
4,0,140000.0,140250.0,165500.0,103950.0,145000.0,154000.0,78.918228,78.918228,81.598080,...,5408.0,5408.0,5408.0,5408.0,2492.0,2492.0,2492.0,205414.599202,205012.252913,203781.903446
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122,0,602750.0,555000.0,519000.0,542500.0,540000.0,525000.0,234.281690,230.061350,218.750000,...,5713.0,5713.0,5713.0,5713.0,2728.0,2728.0,2728.0,499412.241077,495606.636012,488436.530983
123,0,568750.0,602750.0,555000.0,455000.0,542500.0,540000.0,232.192450,234.281690,230.061350,...,5713.0,5713.0,5713.0,5713.0,2728.0,2728.0,2728.0,500013.225078,499412.241077,495606.636012
124,0,570000.0,568750.0,602750.0,472642.5,455000.0,542500.0,225.866425,232.192450,234.281690,...,5713.0,5713.0,5713.0,5713.0,2728.0,2728.0,2728.0,498003.211865,500013.225078,499412.241077
125,0,572500.0,570000.0,568750.0,460285.0,472642.5,455000.0,231.490967,225.866425,232.192450,...,5713.0,5713.0,5713.0,5713.0,2728.0,2728.0,2728.0,496887.003144,498003.211865,500013.225078


In [ ]:
#ts_train.loc[120:135,['price_1m', 'zipcode_1m']] # Check on the data.

In [127]:
X_train = ts_train.drop(columns=['target_profitable_3m'])
y_train = ts_train['target_profitable_3m']

In [128]:
X_train.shape, y_train.shape #, y_train, y_train.value_counts()

((790702, 108), (790702,))

In [129]:
X_test = ts_test.drop(columns=['target_profitable_3m'])
y_test = ts_test['target_profitable_3m']

In [133]:
from sklearn.linear_model import SGDClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import precision_score, confusion_matrix

In [131]:
scaler = MinMaxScaler()

In [ ]:
scaler.fit(X_train)

,feature_range,"(0, ...)"
,copy,True
,clip,False


In [ ]:
X_train_scaled = scaler.transform(X_train)

In [ ]:
X_test_scaled = scaler.transform(X_test)

In [137]:
model = SGDClassifier(loss='log_loss')

In [138]:
model.fit(X_train_scaled, y_train)

,loss,'log_loss'
,penalty,'l2'
,alpha,0.0001
,l1_ratio,0.15
,fit_intercept,True
,max_iter,1000
,tol,0.001
,shuffle,True
,verbose,0
,epsilon,0.1
,n_jobs,None


In [145]:
y_pred = model.predict(X_test_scaled)

In [146]:
model.score(X_test_scaled, y_test)

0.7069279366099154

In [147]:
precision_score(y_test, y_pred)

0.2568582154201559

In [ ]:
y_pred_proba = model.predict_proba(X_test_scaled) # The user should get the probability of the suggestion of the model.
y_pred_proba

array([[0.72522592, 0.27477408],
       [0.71444495, 0.28555505],
       [0.70617656, 0.29382344],
       ...,
       [0.55358893, 0.44641107],
       [0.64388889, 0.35611111],
       [0.70051328, 0.29948672]], shape=(56034, 2))

In [149]:
confusion_matrix(y_test, y_pred)

array([[37833,  5147],
       [11275,  1779]])

SGD Classifier look on the precision on the test set.
Gradient boosting classifier.

XGBoost classifier.

-> Target might be changed to a regression where we predict the price of the zip code.

-> Convert zipcode into longitude and latitude if possioble. -> Advantage: The model can differenciate the distance between the cities.

Scale with min max scaler as always not.

For final product:

1. Convert zipcodes (delete) into coordinates (add two colums) -> update data frame.  -> check which data set works better.
2. Compare SGDClassifier with Gradient boosting classifier. -> also here with both data sets. -> Decision what to use and may try XGBoost classifier as well (optional).
3. Create another train and test set -> train all but the last month in 2023, test = last month 2023. -> Take care of the dimensions.
4. Create a dataframe with the predictions -> y_pred = model.predict(X_test_scaled) -> one result per zip code. -> After this I can use to start the API.
5. Execute data analytics on it -> where are profitable zipcodes where are not? -> plots, plots, plots!
6. User wants do know in which zipcode a profitable investment is likely. -> Close to a specific zipcode e. g. a new employer.
